# Reviewing model versions

## Introduction

This cookbook will guide you through the handling and comparison of an existing computational model in jinko.  
In particular, you will be able to retrieve two specified model versions and output an html report containing the differences  


Linked resources: [Jinko](https://jinko.ai/project/e0fbb5bb-8929-439a-bad6-9e12d19d9ae4?labels=59a37e3f-471f-4958-a519-aa9ee128f5f7).

In [ ]:
# Jinko specifics imports & initialization

import sys

sys.path.insert(0, "../lib")
import jinko_helpers as jinko

# Connect to Jinko (see README.md for more options)

jinko.initialize()

In [ ]:
import zipfile
import io
from typing import List, Any
import json
from deepdiff import DeepDiff
import difflib
import re

# List all models in a particular folder
folderId = "59a37e3f-471f-4958-a519-aa9ee128f5f7"

# Make a request to get all project items in the specified folder and filter out only ComputationalModel items
models: List[Any] = jinko.makeRequest(f'/app/v1/project-item?folderId={folderId}').json()
models = [m for m in models if m['kind'] == "ComputationalModel"] 

# Create dictionaries to map model names to their corresponding SID and core ID
sid_dict = {m['name']: m['sid'] for m in models}
model_dict = {m['name']: m['coreId']['id'] for m in models}

# Local folder path to store downloaded model versions
localFolderPath = "outputs"

# Function to format the diff output
def diffFormatCreator(m, a, b):
    result = ""
    delete = "<span style='color:red;'>"
    end = "</span>"
    insert = "<span style='color:green;'>"
    
    # Loop through the opcodes and format the changes
    for tag, i1, i2, j1, j2 in m.get_opcodes():
        if tag == 'replace':
            result += f' {delete}{" ".join(a[i1:i2])}{end} '
            result += f'{insert}{" ".join(b[j1:j2])}{end} '
        if tag == 'delete':
            result += delete
            result += ' '.join(a[i1:i2])
            result += end
        if tag == 'insert':
            result += insert
            result += ' '.join(b[j1:j2])
            result += end
        if tag == 'equal':
            result += ' '.join(a[i1:i2])
    return result

# Function to extract and format the diff keys
def formatKey(input_str, componentsHere):
    # Remove 'root' and split by the brackets
    parts = re.split(r"root|\[|\]|'|specifics|contents", input_str)

    # Get the component ID based on the index
    immutableIdHere = parts[3]
    idHere = [component for component in componentsHere if component['immutableId']==immutableIdHere][0]['id']
    
    # Filter out empty strings and join the remaining parts with spaces
    formatted_str = idHere + ' ' + ' '.join(filter(None, parts[4:]))

    return formatted_str

# Function to review two versions of a model
def reviewVersions(modelName, modelId, newVersion = 0, backTrack = 1):
    # List all model versions for the specified model
    sid = sid_dict[modelName]
    versions: List[Any] = jinko.makeRequest(f'/app/v1/project-item/{sid}/versions?onlyLabeled=true').json()
    
    # Get the latest and one previous version
    if newVersion < 0 or backTrack <1:
        print( f'newVersion needs to be at least 0 and backTrack needs to be at least 1')
        return
    if newVersion+backTrack+1 > len(versions):
        print( f'backTrack is too high, there is only {len(versions)-newVersion-1} older versions available')
        return
    
    version_new = versions[newVersion]
    version_old = versions[newVersion + backTrack]
    
    # Extract snapshot IDs and labels
    snapshotId_new = version_new['coreId']['snapshotId']
    snapshotId_old = version_old['coreId']['snapshotId']
    label_new = version_new['label']
    label_old = version_old['label']

    print(f'Comparing new version {label_new} and old version {label_old} of {modelName}')
    
    # Download the model versions
    model_new = jinko.makeRequest(f'/core/v2/model_editor/jinko_model/{modelId}/snapshots/{snapshotId_new}').json()
    model_old = jinko.makeRequest(f'/core/v2/model_editor/jinko_model/{modelId}/snapshots/{snapshotId_old}').json()
    
    # Save the model versions as JSON files
    with open(f'{localFolderPath}/{modelName}-{label_new}.json', 'w') as f:
        json.dump(model_new, f, indent=4)
    with open(f'{localFolderPath}/{modelName}-{label_old}.json', 'w') as f:
        json.dump(model_old, f, indent=4)

    # Generate tags for the HTML diff
    tagsHere = ('<br>'.join([f'{tag["id"]}: {tag["immutableId"]}' for tag in model_new['tags']]))
    html_diff = "Tags:<br>" + tagsHere + "<br><br>"
    html_diff += f'Comparing new version {label_new} and old version {label_old} of {modelName}<br>'
    
    # Compare components and generate HTML diff for each key
    for key in model_new['components'].keys():
        oldComponents = model_old['components'][key]
        newComponents = model_new['components'][key]
        for i in oldComponents:
            del i['diagnostics']
        for i in newComponents:
            del i['diagnostics']      
        # Use DeepDiff to find differences excluding diagnostics
        diffDict = DeepDiff(oldComponents, newComponents, exclude_regex_paths="root\[\d+\]\['diagnostics'\]", ignore_order=True, group_by='immutableId')
        # Check if there are any differences
        if diffDict != {}:
            html_diff += f'<h2>Changes in {key} </h2>'
            # Handle changed values
            if 'values_changed' in diffDict.keys(): 
                for idHere in diffDict['values_changed']:
                    old_value = str(diffDict['values_changed'][idHere]['old_value'])
                    new_value = str(diffDict['values_changed'][idHere]['new_value'])
                    m = difflib.SequenceMatcher(a=old_value.split(), b=new_value.split())
                    changes = diffFormatCreator(m, old_value.split(), new_value.split())
                    html_diff += f'Changes in {formatKey(idHere, oldComponents)}:<br>{changes}<br><br>'
                    
            # Handle added items
            if 'dictionary_item_added' in diffDict.keys(): 
                for idHere in diffDict['dictionary_item_added']:
                    clean_id = re.sub(r"root|\[|\]|'", "",idHere)
                    new_value = [component for component in newComponents if component['immutableId']==clean_id][0]
                    json_new_value = str(json.dumps(new_value, indent=4))
                    escaped_json_str = json_new_value.replace('\n', '<br>').replace(' ', '&nbsp;')
                    html_output = f"<span style='color:green;'>{escaped_json_str}</span>"
                    html_diff += f'Added {formatKey(idHere, newComponents)}:<br>{html_output}<br><br>'
                    
            # Handle removed items
            if 'dictionary_item_removed' in diffDict.keys(): 
                for idHere in diffDict['dictionary_item_removed']:
                    clean_id = re.sub(r"root|\[|\]|'", "",idHere)
                    old_value = [component for component in oldComponents if component['immutableId']==clean_id][0]
                    json_old_value = str(json.dumps(old_value, indent=4))
                    escaped_json_str = json_old_value.replace('\n', '<br>').replace(' ', '&nbsp;')
                    html_output = f"<span style='color:red;'>{escaped_json_str}</span>"
                    html_diff += f'Removed {formatKey(idHere, oldComponents)}:<br>{html_output}<br><br>'
            
        # Save the HTML diff
        with open(f'{localFolderPath}/{modelName}-{label_old}-{label_new}-diff.html', 'w') as f:
            f.write(html_diff)

# Example usage
reviewVersions('Test-Model', model_dict['Test-Model'])
reviewVersions('Test-Model', model_dict['Test-Model'], backTrack = 5)
reviewVersions('Test-Model', model_dict['Test-Model'], newVersion = 3, backTrack = 1)
reviewVersions('Test-Model', model_dict['Test-Model'], newVersion = 5, backTrack = 1)
